In [1]:
!pip install transformers
!pip install bs4
!pip install requests

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
import os
from google.colab import userdata

In [2]:
api_key = userdata.get('API_KEY')
cse_id = userdata.get('CSE_ID')
access_token = userdata.get('ACCESS_TOKEN')

In [3]:
question = """
system: you are a summarizer that finds the answer the user is looking for in the context provided by the google search results top links and your previous conversation with the user. answer in atleast 100 words.
user: {prompt}
"""

In [33]:
qa_pipeline = pipeline('question-answering', model='deepset/roberta-large-squad2', token=access_token)

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [10]:
def google_search(query, api_key, cse_id, num_results=3):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": api_key,
        "cx": cse_id,
        "num": num_results
    }
    response = requests.get(url, params=params)
    results = response.json().get('items', [])
    return results

In [39]:
def get_context(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = ""
    paragraphs = "".join([p.get_text() for p in soup.find_all('p')[:2]])
    return paragraphs

In [40]:
def get_answer(prompt):
    top_url = [result['link'] for result in google_search(prompt, api_key, cse_id, num_results=3)]
    print(top_url, "\n\n")
    context = r"".join([get_context(url) for url in top_url])
    print(context, "\n\n")
    que = question.format(prompt=prompt)
    print(que, "\n\n")
    answer = qa_pipeline(question=que, context=context)
    return answer

In [42]:
answer = get_answer("size of indian army")
print(answer['answer'])

['https://en.wikipedia.org/wiki/Indian_Army', 'https://www.nam.ac.uk/explore/armies-east-india-company', 'https://en.wikipedia.org/wiki/Indian_Armed_Forces'] 



1 April 1895; 129 years ago (1895-04-01) (as Indian Army)
The Indian Army is the land-based branch and largest component of the Indian Armed Forces. The President of India is the Supreme Commander of the Indian Army,[3] and its professional head is the Chief of Army Staff (COAS). The Indian Army was established on 1 April 1895 alongside the long established presidency armies of the East India Company, which too were absorbed into it in 1903. Some princely states maintained their own armies which formed the Imperial Service Troops which, along with the Indian Army formed the land component of the Armed Forces of the Crown of India, responsible for the defence of the Indian Empire.[4][5] The Imperial Service Troops were merged into the Indian Army after independence. The units and regiments of the Indian Army have diverse histor